In [2]:
import numpy as np

## Nash equilibrium code

In [1]:
def nash_eq(d0, a1l, a1h, a2l, a2h, a3l, a3h, e, p, disc, n, T):    
    def previous_n(n, z, e): 
        n_prev = (n - e * z) / (1 - e)
        if n_prev < 0:
            return 0
        elif n_prev > 1:
            return 1
        else:
            return n_prev

    def sum_constants_fun(e, disc, p, T):
        num_periods = np.max([T, 50]) # use 50 generations to approximate infinity if T < 50
        values = np.ones(num_periods)
        for i in range(1, num_periods+1):
            # values[i-1] = (1 - e) ** i * e / (1 + disc * i)       # --> hyperbolic discounting
            values[i-1] = (1 - e) ** (i - 1) * e * disc ** i / p    # --> exponential discounting
        return values

    def z_bestresponse_fun(d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants, n, z_known, z_longrun):
        num_periods = len(sum_constants)
        z_filtered = z_known[p-2:][::p]
        num_remaining = num_periods - len(z_filtered)
        all_z = np.append(z_filtered, np.ones(num_remaining) * z_longrun)
        weighted_coefficients = np.ones(num_periods) * a2h - np.ones(num_periods) * (a2h-a2l + a3h * (p-1)/p -a3l/p) * all_z - all_z * all_z * (a3h-a3l) * (p-1)/p 
        c = np.sum(sum_constants * weighted_coefficients)
        term = d0 + (a2h-a2l) * n - a1l/p - a3l * n / p - c 
        z = -term * p / ((p - 1) * (a1h - a1l + (a3h - a3l) * n)) 
        return np.clip(z, 0, 1)

    def z_longrun_fun(d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants_inf):
        if a3l == a3h:
            num = a1l / p + a2h * sum_constants_inf - d0
            den = (1 + sum_constants_inf) * (a2h - a2l - a3l/p) + (a1h - a1l) * (p - 1) / p + a3h * (p-1)/p * sum_constants_inf + 1e-7
            zlr = np.array([num/den, num/den])
        else:
            b1 = -(1+sum_constants_inf) * (p * (a2h-a2l) - a3l) - sum_constants_inf * a3h * (p-1) - (a1h-a1l) * (p-1)
            b2 = 4 * (1+sum_constants_inf) * (p-1) * (a3h-a3l) * (d0 + sum_constants_inf * a2h - a1l/p)
            b3 = 2 * (1+sum_constants_inf) * (p-1) * (a3h-a3l) + 1e-7
            zlr = np.array([(b1 + np.sqrt(np.max([b1 ** 2 - p * b2,0]))) / b3, (b1 - np.sqrt(np.max([b1 ** 2 - p * b2,0]))) / b3])
        return np.clip(zlr, 0 ,1)
    
    def nash_path(z_longrun, d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants):
        n = np.array([z_longrun])
        z = np.array([z_longrun])

        t = 0
        while (z[0] > 0) and (t < T*p):
            if t < p: 
                z = np.append(z_bestresponse_fun(d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants, n[0]+1e-7, z[:-1], z_longrun), z)
            else:
                z = np.append(z_bestresponse_fun(d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants, n[0], z[:-1], z_longrun), z)
            n = np.append(previous_n(n[0], z[0], e/p), n)
            t += 1

        while (n[0] < 1) and (t < T*p):
            z = np.append(0, z)
            n = np.append(previous_n(n[0], 0, e/p), n)
            t += 1

        diff = T*p-t
        if diff > 0:
            n = np.append(n, np.ones(diff) * z_longrun)
            z = np.append(z, np.ones(diff) * z_longrun)
        return n, z

    sum_constants = sum_constants_fun(e, disc, p, T)
    sum_constants_inf = np.sum(sum_constants)
    z_longrun1, z_longrun2 = z_longrun_fun(d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants_inf) 
    #n1, z1 = nash_path(z_longrun1, d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants)
    #n2, z2 = nash_path(z_longrun2, d0, a1l, a1h, a2l, a2h, a3l, a3h, p, sum_constants)

    return z_longrun1 * np.ones(T*p+1), z_longrun1 * np.ones(T*p+1), z_longrun2 * np.ones(T*p+1), z_longrun2 * np.ones(T*p+1) #n1, z1, n2, z2

#### Avergage (over time) KL-divergence

In [ ]:
def jensen_shannon_divergence(me, ne):
    M = [(me + ne) / 2 + 1e-7, (2 - me - ne) / 2 + 1e-7] 
    element1 = me * np.log(me/M[0]+1e-7) + ne * np.log(ne/M[0]+1e-7)
    element2 = (1-me) * np.log((1-me)/M[1]+1e-7) + (1-ne) * np.log((1-ne)/M[1]+1e-7)
    return (element1 + element2) / 2

def ave_kl(zl_evolution, ne_evolution):
    sum = 0
    for i in range(len(zl_evolution)):
        sum += jensen_shannon_divergence(zl_evolution[i], ne_evolution[i])
    return (-1/len(zl_evolution) * sum) #'%.04f' % (-1/len(zl_evolution) * sum)